We are finally using the SFP algorithm in order to avoid the use of a universe list with all possible elements which creates a lot of false signals. It uses all the algorithms from the previous version, in addition, it uses SFP version on the top of the previous algorithms to give the really most frequent words, instead of a lot of fake counts.Unlike Apple's case, we will kinda maintain length 5 strings and will pad chars if not full.

In [20]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter
from operator import itemgetter

In [21]:
class Differential_Privacy_CMS_with_SFP():
    def __init__(self, width, depth, epsilon, max_coor=None):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = np.random.choice(np.arange(0,self.width),replace=False,size=self.depth) if width!=None else None
        self.max = max_coor

    def Client_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j)
        
    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 
        
    def Server_Side(self,Sketch_Matrix,d,length):
        '''It returns the estimated frequency of a data element given to it. So it has two parameters, data element and the length of the 
        data stream we are considering'''
        n = length 
        frac1 = self.width/(self.width-1)
        frac2 = n/self.width
        row_sum = 0
        for i in range(self.depth):
            hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
            int_hex = int(hex_num[:5],16)
            index = int_hex % self.width
            row_sum = row_sum + Sketch_Matrix[i,index]
        avg_row_sum = row_sum/self.depth 
        subtraction = avg_row_sum - frac2 
        assumed_freq = frac1*subtraction 
        return assumed_freq

    def Count_Mean_Sketch(self,client_side_result_list,Universe_list): 
        '''D_s is the stream of data and this is a subset of universe of data'''
        Sketch_Matrix = []
        freq_vect = {}
        length = len(client_side_result_list) 
        Sketch_Matrix = self.Compute_Sketch_Matrix(client_side_result_list)
        for d in Universe_list:
            freq_vect[d] = self.Server_Side(Sketch_Matrix,d,length) 
        return freq_vect

def Client_SFP(d,CMS_class_main, CMS_class_not):
    '''d is a tuple in our example, so for converting it to a string, we are doing this d[0]*self.max+d[1] and then make it a string.'''
    l = np.random.randint(2)
    d =  str(d[0]*90+d[1])
    if len(d)<4:
        pad = 4-len(d)
        for i in range(pad):
            d+='0'
    output256 = hashlib.sha256(d.encode('utf-8')).hexdigest()#h(s)
    r = output256 + d[2*l:2*l+2]
    return (CMS_class_not.Client_Side(r), CMS_class_main.Client_Side(d),l,r)

def splitter(string):
    '''getting a string, it splits it into two parts, second part contains the last two characters.'''
    return string[:-2],string[-2:]

def Server_SFP(M,M1,M2,T):
    '''It gets the output from client frequency cunter, which is basically three frequency histogram dictionaries, then it does the next steps.'''
    Q1=set(dict(sorted(M1.items(), key = itemgetter(1), reverse =True)[:T]).keys())
    Q2=set(dict(sorted(M2.items(), key = itemgetter(1), reverse =True)[:T]).keys())
    Q1_dict={}
    Q2_dict={}
    for elt in Q1:
        word,val = splitter(elt)
        Q1_dict[word]=val
    for elt in Q2:
        word,val=splitter(elt)
        Q2_dict[word]=val 
    #will find the intersection of Q1 and Q2
    new_string_set = set()
    for word in Q1_dict:
        if word in Q2_dict:
            add_string = Q1_dict[word]+Q2_dict[word]
            new_string_set.add(add_string)
    final_freq_dict ={}
    for elt in new_string_set:
        final_freq_dict[elt]=M[elt]
    return final_freq_dict

def Client_frequency_counter_on_average(run_number,D_s,Universe_list):#this part is controlled by client side too
    '''cal'''
    b_freq_cum={d:0 for d in Universe_list}
    Universe_list1=[]
    for i in range(run_number):
        big_elt_set=[]
        CMS_Class_Main =  Differential_Privacy_CMS_with_SFP(100,100,5,90)
        CMS_Class_not = Differential_Privacy_CMS_with_SFP(100,100,5,90)
        for elt in D_s:
            val = Client_SFP(elt,CMS_Class_Main,CMS_Class_not)
            big_elt_set.append(val)
        beta_data=[]
        alpha_list1=[]
        alpha_list2=[]
        for elt in big_elt_set:
            beta_data.append(elt[1])
            Universe_list1.append(elt[3])
            if elt[2]==0:
                alpha_list1.append(elt[0])
            else:
                alpha_list2.append(elt[0])
        a_freq_cum1={d:0 for d in Universe_list1}
        a_freq_cum2={d:0 for d in Universe_list1}
        #created lists of client side.
        b_freq_dict = CMS_Class_Main.Count_Mean_Sketch(beta_data,Universe_list)
        a_freq_dict1 = CMS_Class_not.Count_Mean_Sketch(alpha_list1,Universe_list1)
        a_freq_dict2 = CMS_Class_not.Count_Mean_Sketch(alpha_list2, Universe_list1)
        for d in b_freq_dict:
            b_freq_cum[d]=(i*b_freq_cum[d]+b_freq_dict[d])/(i+1)   
        for d in Universe_list1:
            a_freq_cum1[d]=(i*a_freq_cum1[d]+a_freq_dict1[d])/(i+1)
            a_freq_cum2[d]=(i*a_freq_cum2[d]+a_freq_dict2[d])/(i+1)
    return b_freq_cum, a_freq_cum1, a_freq_cum2

def Sequence_Fragment_Puzzle(run_number,D_s,T,Universe_list):
    '''Main driver code of sequence fragment puzzle.'''
    M, M1, M2 = Client_frequency_counter_on_average(run_number,D_s,Universe_list)
    return Server_SFP(M, M1, M2, T)

In [ ]:
#plan: we need to call client side multiple times becuase else we don't get different distribution. For one 
#call of client side, we compute sketch matrix and do frequency count. but the client side is being called from client SFP.
#so in order to average like something, we need to call client side multiple times too. And the client side algo is being callled from driver 
#SFP. To sum up, we need to do this process multiple times and take the average to get a good count-
#1.Call Client SFP
#2.Push this output to the Sketch CMS matrix
#3.Push Sketch CMS Matrix to the server for frequency histogram
#4.Continue this process multiple times  or repeat this process multiple times(So basically the client device does this query multiple times)
#to get a better frequency estimation)
#5. Then send this frequency histograms to the server SFP to do further calculation.

In [22]:
true_path = [(15, 7),
 (15, 6),
 (15, 7),
 (14, 7),
 (14, 6),
 (13, 6),
 (13, 5),
 (14, 5),
 (15, 5),
 (14, 5),
 (13, 5),
 (12, 5),
 (12, 6),
 (12, 5),
 (12, 6),
 (13, 6),
 (12, 6),
 (12, 5),
 (12, 6),
 (12, 5),
 (12, 4),
 (12, 3),
 (13, 3),
 (13, 2),
 (12, 2),
 (12, 1),
 (12, 0),
 (11, 0),
 (10, 0),
 (9, 0),
 (8, 0),
 (9, 0),
 (9, 1),
 (10, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (13, 1),
 (12, 1),
 (12, 2),
 (12, 3),
 (11, 3),
 (11, 2),
 (12, 2),
 (13, 2),
 (12, 2),
 (11, 2),
 (10, 2),
 (9, 2),
 (10, 2),
 (11, 2),
 (11, 3),
 (10, 3),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 7),
 (9, 7),
 (9, 8),
 (8, 8),
 (7, 8),
 (7, 9),
 (7, 8),
 (6, 8),
 (6, 9),
 (6, 10),
 (5, 10),
 (5, 9),
 (6, 9),
 (6, 10),
 (6, 11),
 (5, 11),
 (6, 11),
 (6, 10),
 (5, 10),
 (4, 10),
 (4, 9),
 (5, 9),
 (5, 8),
 (4, 8),
 (4, 9),
 (4, 8),
 (4, 7),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 9),
 (7, 9),
 (6, 9),
 (7, 9),
 (6, 9),
 (5, 9),
 (4, 9),
 (5, 9),
 (5, 8),
 (4, 8),
 (3, 8),
 (2, 8),
 (1, 8),
 (2, 8),
 (2, 9),
 (1, 9),
 (2, 9),
 (2, 10),
 (3, 10),
 (2, 10),
 (2, 11),
 (3, 11),
 (2, 11),
 (1, 11),
 (2, 11),
 (1, 11),
 (1, 10),
 (1, 9),
 (2, 9),
 (1, 9),
 (2, 9),
 (3, 9),
 (2, 9),
 (2, 8),
 (1, 8),
 (1, 9),
 (2, 9),
 (1, 9),
 (1, 10),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 11),
 (2, 11),
 (1, 11),
 (0, 11),
 (1, 11),
 (1, 12),
 (1, 11),
 (1, 10),
 (1, 9),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 11),
 (3, 11),
 (3, 10),
 (3, 9),
 (2, 9),
 (2, 10),
 (1, 10),
 (1, 9),
 (2, 9),
 (2, 10),
 (2, 11),
 (3, 11),
 (3, 10),
 (4, 10),
 (4, 9),
 (3, 9),
 (4, 9),
 (4, 8),
 (5, 8),
 (4, 8),
 (5, 8),
 (4, 8),
 (4, 9),
 (4, 8),
 (5, 8),
 (5, 7),
 (5, 6),
 (5, 5),
 (5, 4),
 (5, 3),
 (5, 4),
 (6, 4),
 (7, 4),
 (8, 4),
 (7, 4),
 (8, 4),
 (8, 3),
 (7, 3),
 (8, 3),
 (8, 2),
 (9, 2),
 (8, 2),
 (7, 2),
 (7, 3),
 (6, 3),
 (5, 3),
 (4, 3),
 (4, 2),
 (5, 2),
 (4, 2),
 (3, 2),
 (3, 1),
 (2, 1),
 (2, 0),
 (2, 1)]

In [23]:
universe_list = [(x,y) for x in range(20) for y in range(80)]


In [24]:
def string_maker(d):
    d =  str(d[0]*90+d[1])
    if len(d)<4:
        pad = 4-len(d)
        for i in range(pad):
            d+='0'
    return d

In [25]:
Universe_list=[]
for elt in universe_list:
    Universe_list.append(string_maker(elt))
    


In [ ]:
Sequence_Fragment_Puzzle(500,true_path,250,Universe_list)